# FCR GENERAL

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
import pandas as pd
from pathlib import Path

def calcular_reiteraciones(input_path, output_path=None, window_days=7, sep=";"):
    """
    Agrega campos de reiteración sin eliminar filas:
      - FECHA_SIGUIENTE
      - DIAS_HASTA_REITERACION          (entero; NaN si no hay siguiente)
      - HORAS_HASTA_REITERACION         (float; NaN si no hay siguiente)
      - FLAG_REITERACION_24   -> 1 si la reiteración ocurre en >0h y <=24h
      - FLAG_REITERACION_7D   -> 1 si la reiteración ocurre en 1..window_days días (incluyente)
      - FLAG_REITERACION_30D  -> 1 si la reiteración ocurre en (window_days+1)..29 días (incluyente)

    Parámetros
    ----------
    input_path : str
        Ruta del CSV de entrada.
    output_path : str | None
        Si se entrega, guarda el resultado en esa ruta.
    window_days : int
        Límite superior para el flag de 7 días (por defecto 7).
    sep : str
        Separador del CSV (por defecto ';').

    Retorna
    -------
    pandas.DataFrame
        DataFrame con las columnas agregadas.
    """

    # --- 1) Leer base
    df = pd.read_csv(input_path, sep=sep)

    # Validaciones mínimas
    if "CustomerIdentity" not in df.columns:
        raise KeyError("No se encontró la columna 'CustomerIdentity' en el archivo de entrada.")
    if "StorageDate" not in df.columns:
        raise KeyError("No se encontró la columna 'StorageDate' en el archivo de entrada.")

    # --- 2) Parseo robusto de StorageDate (acepta offsets tipo -03:00)
    df["StorageDate"] = pd.to_datetime(df["StorageDate"], errors="coerce", utc=False)
    # Si viene con tz, quitarla para trabajar en naive (local)
    try:
        df["StorageDate"] = df["StorageDate"].dt.tz_localize(None)
    except (TypeError, AttributeError):
        pass  # ya era naive

    # --- 3) Orden y cálculo de siguiente fecha por cliente
    df = df.sort_values(["CustomerIdentity", "StorageDate"]).reset_index(drop=True)
    df["FECHA_SIGUIENTE"] = df.groupby("CustomerIdentity")["StorageDate"].shift(-1)

    # --- 4) Diferencias en días y horas
    delta = df["FECHA_SIGUIENTE"] - df["StorageDate"]
    df["DIAS_HASTA_REITERACION"] = delta.dt.days
    df["HORAS_HASTA_REITERACION"] = delta.dt.total_seconds() / 3600

    # --- 5) Flags INCLUYENTES (pueden solaparse; cascada 24 -> 7D -> 30D)
    df["FLAG_REITERACION_24"] = 0
    df["FLAG_REITERACION_7D"] = 0
    df["FLAG_REITERACION_30D"] = 0

    # Condiciones base
    cond_24 = (df["HORAS_HASTA_REITERACION"] > 0) & (df["HORAS_HASTA_REITERACION"] <= 24)
    cond_7d = (df["DIAS_HASTA_REITERACION"] >= 1) & (df["DIAS_HASTA_REITERACION"] <= int(window_days))
    # Docstring indica hasta 29 días
    cond_30d = (df["DIAS_HASTA_REITERACION"] >= int(window_days) + 1) & (df["DIAS_HASTA_REITERACION"] <= 30)

    # Asignación incluyente en cascada
    df.loc[cond_24, ["FLAG_REITERACION_24", "FLAG_REITERACION_7D", "FLAG_REITERACION_30D"]] = 1
    df.loc[cond_7d & ~cond_24, ["FLAG_REITERACION_7D", "FLAG_REITERACION_30D"]] = 1
    df.loc[cond_30d & ~cond_7d & ~cond_24, "FLAG_REITERACION_30D"] = 1

    # --- 6) Guardar si corresponde
    if output_path:
        Path(output_path).parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(output_path, index=False, sep=sep)

    return df

# ======================
# EJEMPLO DE USO
# ======================
if __name__ == "__main__":
    INPUT = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha.csv"
    OUTPUT_CSV = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Reiteracion General Consolidada.csv"

    df_resultado = calcular_reiteraciones(
        input_path=INPUT,
        output_path=OUTPUT_CSV,
        window_days=7,
        sep=","  # ¡ojo con el delimitador real de tu archivo!
    )


C:\Users\wduran\AppData\Local\Temp\ipykernel_19968\4093002841.py:32: DtypeWarning: Columns (17,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,171,172,173,174,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,202,203) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path, sep=sep)


# FCR NO VOZ VTR

In [30]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Reiteracion General Consolidada.csv"
base_fcr = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'

# Ajustar las opciones para mostrar todas las columnas
pd.set_option('display.max_columns', None)

C:\Users\wduran\AppData\Local\Temp\ipykernel_19968\2060631343.py:3: DtypeWarning: Columns (17,63,64,67,68,69,70,73,74,75,76,77,78,79,80,81,82,84,85,86,87,88,89,90,103,108,109,110,111,112,113,114,115,116,117,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,171,172,173,174,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,202,203) have mixed types. Specify dtype option on import or set low_memory=False.
  base_fcr = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'


In [31]:
base_fcr.sample(1)

,SequentialId,CustomerIdentity,AgentIdentity,Status,StorageDate,ExpirationDate,CloseDate,Team,Closed,Tags,QueueTime,FirstResponseTime,AverageResponseTime,CustomerName,CustomerEmail,CustomerGender,CustomerCity,CustomerPhoneNumber,AgentName,AgentEmail,1,2,3,campaignOriginator,botOrigin,campaignMessageTemplate,campaignId,Menu Hogar,priorityTag,Campaign,Menu Principal,team,rut,activeMessageFileName,actividad,activa,headline,body,source_type,source_id,source_url,media_id,tiporedproductotoa,08-oct,13:00-16:00horas,NumeroSerie,30-ago,16:00-19:00horas,SourceId,SourceUrl,SourceTitle,SourceType,25-sept,direccion,nombreplanprincipal,svatotal,databeneficiosincluidos,preciopermanente,dctomensual,montomensualplan,totalproductoadicional,07-oct,10:00-13:00horas,21-oct,oct-13,27-sept,variable1,23-oct,13:00-16:00,09-oct,20-nov,nombre,mes,montopago,numboleta,fechapago,fechacorte,11-oct,1db07bec29ded479f93733c484b4cb7ef,cargomensual,montomensual,23-ago,16:00-19:00,4,5,6,7,28-ago,urlarr[1],10-jul,14-feb,Fecha,HORA,INTERVALO,NEUTRA,Semana,Def_IBR,Llave,Flag_Outbound,MAIL,Flag_Dotacion_Konecta,Rama,Defle_meta,fecha,access_token,activity_id,appt_number,number,img,meta_ad_id,meta_ad_headline,meta_ad_type,meta_ad_url,medios,15-abr,17-abr,rut_cliente,imagen,monto,numero,21-abr,medio,Template,menuPrincipal,ticketCerrado,06462daba990c432787bac8e9a54eaede,10,name,8,9,0,sept-13,10f190d419317495ab3c4c114071efcf3,16-abr,marca,montofinpegado,montofin,1c1c94c44c781416fa3c32a9d2661d779,17589781ee1e94b95b6592b9815b9ae6d,url,product,productprecio,plan,cargo,desc,cargofijo,sva,valor,servicio,subflowId,13a8485c130924cc9a875768611091981,1d10645a7743548b7b7c18e5250ba9c2e,number6b1b088dfa84473bbd7d857d6ac847b3,1c9af89a06f3b4cd7b33862bed4279493,1105e9e6f22924ee2950572a0fb3806a3,15c5e4496bdd344d8b3e12dd1a45b6c5a,18e6db50c07ec4633a69a59407fa551d0,18735621ed8d549f894d85c3cf1b9f891,18-abr,12689ce9218a249328e6d871f3b3507e9,191963c9f901c463586337e881bdb560e,1c6ee6fe3ddd74cb6ab13aea6ca60cb3f,1dacb0a0dee4d4db69985fbd4ea1ee9f8,glosa,1280a404ecf174855a17e89ec8a0a8b84,1aae9e0406197455ab72f85858f863402,106f438a786624844b5a85fbd67861dc7,19130ce557f4846b3b243bd36f3146693,1398cc9cbb24447a099f3b6f837b33e7d,1e2a78d76f665460783344def9a73ba9a,17092c509dcb64f8a8db006c5a0c824bf,teste1,code,1e9893593ca724de5aa0d9fe14d3125ab,1b6494aee93814727a6a9cbf0753f6422,1dc0ab82bc87a42bdab798b7ec0ad5ad7,1715ef0d6030f4ba2b1ecf00c157a15fc,1492795dce20848adb4c87892b57b990a,16a08f94abb504971aea43c27e7714d7e,activecampaign:6a08f94abb504971aea43c27e7714d7e,19afe09e5006c40ebb7b567b300621dac,1055811656cde4dcfa8353b81ad3a17f0,13f857ef7f52c44388cd9da1d906ff1c9,1809104eca20f446882eeef8d7e3bd9ec,1f20c682451054499930d416609db0b52,medio06daee1056af4e2fba0f6389ecf30adc,18051204300dd4bd38f9e172d1d2b0d7d,16fa3cae479cb4e6caa4c1d5449162b0f,11049111c5e0d45c0ae8d1973a1d6b9cc,178099630d82b44019a5f4bb8cdd61503,176107701c4434b44b649a4d3bef6f7c4,04-jun,activecampaign:fd3b0d1994dc426da152bae6b20b5907,102250dd53a1f4fdd8bf0fdc0c7911b7a,135a2e22fe1dd4267838cc4aa2daeb192,1695a0981a57f4d57bd7ae6414fe399be,1ca434ac0778b4d82aef4ddf16fba7ad0,activecampaign:be235cd8482f43a490f3f810e68d8f90,1faed15d2188c424cae0ecf6a09bffebb,104d9150030c04684bbbe0035c99eeff0,19f2df2c2a1da4fc89623a472d260187c,1017a2cd6316249ca94f556eb8eff5d78,1adcd1b83eb394796bf497c765332b8ab,15b62543c0a2a4a1a84f74f9ec04cd5f8,FECHA_SIGUIENTE,DIAS_HASTA_REITERACION,HORAS_HASTA_REITERACION,FLAG_REITERACION_24,FLAG_REITERACION_7D,FLAG_REITERACION_30D
1854043,2660200,fc28cd30-7212-4747-979d-1b6637ac26b7@tunnel.ms...,arveit.peraltanunez%40e.vtr.cl@blip.ai,ClosedClientInactivity,2025-05-09 13:02:16,NaN,2025-05-09 13:34:54 -04:00,tecnico_fijo,True,"[""inactividad""]",00:00:00,00:00:15,214.0,Carlos Salinas,NaN,NaN,NaN,56998868048,Arveit Peralta,arveit.peraltanunez@e.vtr.cl,NaN,reset,5275918-8,Servicio Hogar,Visita TÃ©cnica,Servicio Hogar,NaN,NaN,NaN,NaN,NaN,NaN,5275918-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [32]:
# Aseguramos que la columna Fecha sea datetime (ya viene en formato YYYY-MM-DD)
base_fcr["Fecha"] = pd.to_datetime(base_fcr["Fecha"], errors="coerce")

# Filtrar solo registros por mes 2025
base_fcr_m = base_fcr[
    (base_fcr["Fecha"].dt.year == 2025) & (base_fcr["Fecha"].dt.month == 7)
]

In [33]:
# Define the file path
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Dotaciones\VTR\Dotación_VTR_20250725.xlsx"
# Read the specific sheet named 'Dotacion' from the Excel file
dotacion = pd.read_excel(archivo, sheet_name='Dotacion')  # Specify the sheet name

In [34]:
dotacion_filtrada = dotacion[
    (dotacion['ESTADO_CONTRATO'].str.lower() == 'vigente') &
    (dotacion['PLATAFORMA'].str.lower().isin(['0 wsp. oopp', '0 wsp postventa'])) &
    (dotacion['CARGO'].str.lower() == 'asesor') &
    (dotacion['ESTADO_ASESOR'].str.lower().isin(['activo', 'licencia', 'vacaciones']))
]

In [35]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
dotacion_filtrada['MAIL'] = dotacion_filtrada['CORREO_ELECTRONICO'].str.split('@').str[0]

C:\Users\wduran\AppData\Local\Temp\ipykernel_19968\3410713443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotacion_filtrada['MAIL'] = dotacion_filtrada['CORREO_ELECTRONICO'].str.split('@').str[0]


In [36]:
# Creamos versiones en minúscula del campo MAIL para el join
dotacion_filtrada['MAIL'] = dotacion_filtrada['MAIL'].str.lower()
base_fcr_m['MAIL'] = base_fcr_m['MAIL'].str.lower()

C:\Users\wduran\AppData\Local\Temp\ipykernel_19968\3461589852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotacion_filtrada['MAIL'] = dotacion_filtrada['MAIL'].str.lower()
C:\Users\wduran\AppData\Local\Temp\ipykernel_19968\3461589852.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_fcr_m['MAIL'] = base_fcr_m['MAIL'].str.lower()


In [37]:
# Hacer merge directo por MAIL
base_fcr_m = base_fcr_m.merge(
    dotacion_filtrada[['MAIL', 'EMPRESA_PAIS']], 
    on='MAIL', 
    how='left'
)

# Crear campo MARCA: 'VTR' si coincide, vacío si no
base_fcr_m['MARCA'] = base_fcr_m['EMPRESA_PAIS'].notna().map({True: 'VTR', False: ''})

In [38]:
# Eliminar filas con NaN en EMPRESA_PAIS
base_fcr_m = base_fcr_m[base_fcr_m['EMPRESA_PAIS'].notna()].copy()

In [39]:
base_fcr_m

,SequentialId,CustomerIdentity,AgentIdentity,Status,StorageDate,ExpirationDate,CloseDate,Team,Closed,Tags,QueueTime,FirstResponseTime,AverageResponseTime,CustomerName,CustomerEmail,CustomerGender,CustomerCity,CustomerPhoneNumber,AgentName,AgentEmail,1,2,3,campaignOriginator,botOrigin,campaignMessageTemplate,campaignId,Menu Hogar,priorityTag,Campaign,Menu Principal,team,rut,activeMessageFileName,actividad,activa,headline,body,source_type,source_id,source_url,media_id,tiporedproductotoa,08-oct,13:00-16:00horas,NumeroSerie,30-ago,16:00-19:00horas,SourceId,SourceUrl,SourceTitle,SourceType,25-sept,direccion,nombreplanprincipal,svatotal,databeneficiosincluidos,preciopermanente,dctomensual,montomensualplan,totalproductoadicional,07-oct,10:00-13:00horas,21-oct,oct-13,27-sept,variable1,23-oct,13:00-16:00,09-oct,20-nov,nombre,mes,montopago,numboleta,fechapago,fechacorte,11-oct,1db07bec29ded479f93733c484b4cb7ef,cargomensual,montomensual,23-ago,16:00-19:00,4,5,6,7,28-ago,urlarr[1],10-jul,14-feb,Fecha,HORA,INTERVALO,NEUTRA,Semana,Def_IBR,Llave,Flag_Outbound,MAIL,Flag_Dotacion_Konecta,Rama,Defle_meta,fecha,access_token,activity_id,appt_number,number,img,meta_ad_id,meta_ad_headline,meta_ad_type,meta_ad_url,medios,15-abr,17-abr,rut_cliente,imagen,monto,numero,21-abr,medio,Template,menuPrincipal,ticketCerrado,06462daba990c432787bac8e9a54eaede,10,name,8,9,0,sept-13,10f190d419317495ab3c4c114071efcf3,16-abr,marca,montofinpegado,montofin,1c1c94c44c781416fa3c32a9d2661d779,17589781ee1e94b95b6592b9815b9ae6d,url,product,productprecio,plan,cargo,desc,cargofijo,sva,valor,servicio,subflowId,13a8485c130924cc9a875768611091981,1d10645a7743548b7b7c18e5250ba9c2e,number6b1b088dfa84473bbd7d857d6ac847b3,1c9af89a06f3b4cd7b33862bed4279493,1105e9e6f22924ee2950572a0fb3806a3,15c5e4496bdd344d8b3e12dd1a45b6c5a,18e6db50c07ec4633a69a59407fa551d0,18735621ed8d549f894d85c3cf1b9f891,18-abr,12689ce9218a249328e6d871f3b3507e9,191963c9f901c463586337e881bdb560e,1c6ee6fe3ddd74cb6ab13aea6ca60cb3f,1dacb0a0dee4d4db69985fbd4ea1ee9f8,glosa,1280a404ecf174855a17e89ec8a0a8b84,1aae9e0406197455ab72f85858f863402,106f438a786624844b5a85fbd67861dc7,19130ce557f4846b3b243bd36f3146693,1398cc9cbb24447a099f3b6f837b33e7d,1e2a78d76f665460783344def9a73ba9a,17092c509dcb64f8a8db006c5a0c824bf,teste1,code,1e9893593ca724de5aa0d9fe14d3125ab,1b6494aee93814727a6a9cbf0753f6422,1dc0ab82bc87a42bdab798b7ec0ad5ad7,1715ef0d6030f4ba2b1ecf00c157a15fc,1492795dce20848adb4c87892b57b990a,16a08f94abb504971aea43c27e7714d7e,activecampaign:6a08f94abb504971aea43c27e7714d7e,19afe09e5006c40ebb7b567b300621dac,1055811656cde4dcfa8353b81ad3a17f0,13f857ef7f52c44388cd9da1d906ff1c9,1809104eca20f446882eeef8d7e3bd9ec,1f20c682451054499930d416609db0b52,medio06daee1056af4e2fba0f6389ecf30adc,18051204300dd4bd38f9e172d1d2b0d7d,16fa3cae479cb4e6caa4c1d5449162b0f,11049111c5e0d45c0ae8d1973a1d6b9cc,178099630d82b44019a5f4bb8cdd61503,176107701c4434b44b649a4d3bef6f7c4,04-jun,activecampaign:fd3b0d1994dc426da152bae6b20b5907,102250dd53a1f4fdd8bf0fdc0c7911b7a,135a2e22fe1dd4267838cc4aa2daeb192,1695a0981a57f4d57bd7ae6414fe399be,1ca434ac0778b4d82aef4ddf16fba7ad0,activecampaign:be235cd8482f43a490f3f810e68d8f90,1faed15d2188c424cae0ecf6a09bffebb,104d9150030c04684bbbe0035c99eeff0,19f2df2c2a1da4fc89623a472d260187c,1017a2cd6316249ca94f556eb8eff5d78,1adcd1b83eb394796bf497c765332b8ab,15b62543c0a2a4a1a84f74f9ec04cd5f8,FECHA_SIGUIENTE,DIAS_HASTA_REITERACION,HORAS_HASTA_REITERACION,FLAG_REITERACION_24,FLAG_REITERACION_7D,FLAG_REITERACION_30D,EMPRESA_PAIS,MARCA
0,3004629,0000974d-9314-4f44-9587-db26f2644541@tunnel.ms...,josh.moreno%40e.vtr.cl@blip.ai,ClosedClientInactivity,2025-07-30 09:19:02,NaN,2025-07-30 09:27:59 -04:00,op_wimt,True,"[""inactividad""]",00:00:00,00:00:08,137.0,Nico Leyton,NaN,NaN,NaN,56990000124,Josh Moreno,josh.moreno@e.vtr.cl,a4a320eb-6de9-4ccd-a7aa-25a223ef9e3d,wimt_tecnico_asignado_v2_,NaN,wimt_tecnico_asignado_v2_,NaN,NaN,op_wimt,NaN,NaN,NaN,"Isla Magdal (sta.fe) 570, Curico","Isla Magdal (sta.fe) 570, Curico",eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9

In [40]:
# Lista de columnas a conservar
cols = [
    "CustomerIdentity", "StorageDate", "FECHA_SIGUIENTE", "DIAS_HASTA_REITERACION",
    "FLAG_REITERACION_24", "FLAG_REITERACION_7D", "FLAG_REITERACION_30D",
    "SequentialId", "AgentIdentity", "Status", "Team",'Tags', "QueueTime",
    "FirstResponseTime", "AverageResponseTime",'CustomerPhoneNumber', "campaignId",'rut',
    "NEUTRA",'Semana', "Def_IBR", "Flag_Outbound", "MAIL",
    "Flag_Dotacion_Konecta", "Defle_meta", "EMPRESA_PAIS", "MARCA"
]

# Filtrar solo esas columnas (las que existan en el DataFrame)
base_fcr_m = base_fcr_m[[c for c in cols if c in base_fcr_m.columns]]


In [41]:
# Ahora sí, guardás el archivo
base_fcr_m.to_csv(r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Base_Consolidada_FCR_JUL.csv", index=False)

# FCR VOZ VTR 

In [42]:
import pandas as pd
import numpy as np
import re
import bisect

In [43]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Base_Consolidada_FCR_JUL.csv"
base_fcr = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'

# Ajustar las opciones para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [44]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Traza Comercial.csv"
traza_comercial = pd.read_csv(archivo, delimiter=';', encoding='latin1')  # Prueba con 'latin1'

In [45]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Traza Comercial 2.csv"
traza_comercial2 = pd.read_csv(archivo, delimiter=';', encoding='latin1')  # Prueba con 'latin1'

In [46]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Traza Comercial 3.csv"
traza_comercial3 = pd.read_csv(archivo, delimiter=';', encoding='latin1')  # Prueba con 'latin1'

In [47]:
# Asegurar conversión de INTERVALO al datetime correcto
traza_comercial3["INTERVALO"] = pd.to_datetime(
    traza_comercial3["INTERVALO"], 
    format="%d-%m-%Y %H:%M",  # porque está como 30-09-2025 23:59
    errors="coerce"
)

# Re-formatear a yyyy-mm-dd HH:MM:SS
traza_comercial3["INTERVALO"] = traza_comercial3["INTERVALO"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [48]:
# Concatenar las bases
traza_comercial_con = pd.concat([traza_comercial,traza_comercial2,traza_comercial3], ignore_index=True)

# Opcional: eliminar duplicados si quieres una consolidación limpia
traza_comercial_con = traza_comercial_con.drop_duplicates()

In [49]:
# Eliminar registros donde ALIAS incluya la palabra "movil" (sin importar mayúsculas/minúsculas)
traza_comercial_con = traza_comercial_con[~traza_comercial_con["ALIAS"].str.contains("movil", case=False, na=False)]

In [50]:
# Convertir INTERVALO a datetime
traza_comercial_con["INTERVALO"] = pd.to_datetime(traza_comercial_con["INTERVALO"], errors="coerce")


# nueva funcion 

In [51]:
import re
import pandas as pd

def _normaliza_rut(serie: pd.Series) -> pd.Series:
    s = serie.astype(str).fillna("").str.upper().str.strip()
    s = s.str.replace(r"[.\s-]", "", regex=True)
    def limpiar(x: str) -> str:
        if not x: return ""
        num, dv = x[:-1], x[-1]
        num = re.sub(r"^0+", "", num) or "0"
        return f"{num}{dv}"
    return s.apply(limpiar)

def _to_datetime_naive(s: pd.Series, dayfirst_hint=None) -> pd.Series:
    if dayfirst_hint is None:
        d = pd.to_datetime(s, errors="coerce")
        m = d.isna()
        if m.any():
            d.loc[m] = pd.to_datetime(s[m], errors="coerce", dayfirst=True)
    else:
        d = pd.to_datetime(s, errors="coerce", dayfirst=dayfirst_hint)
    try:
        d = d.dt.tz_localize(None)
    except Exception:
        try:
            d = d.dt.tz_convert(None)
        except Exception:
            pass
    return d

def add_fcr_voz_flags(
    base_fcr: pd.DataFrame,
    traza_comercial_con: pd.DataFrame,
    rut_col_base: str = "rut",
    rut_col_traza: str = "RUT_CLIENTE",
    fecha_col_base: str = "StorageDate",
    fecha_col_traza: str = "INTERVALO",
    dayfirst_base=None,
    dayfirst_traza=True,
    extra_cols_traza=("RAMAFINAL",)  # <<--- NUEVO: columnas extra de la traza a traer
) -> pd.DataFrame:

    df_base = base_fcr.copy()
    df_traza = traza_comercial_con.copy()

    # Normaliza RUT
    df_base["_RUT"]  = _normaliza_rut(df_base[rut_col_base])
    df_traza["_RUT"] = _normaliza_rut(df_traza[rut_col_traza])

    # Fechas -> datetime naive
    df_base["_T0"]  = _to_datetime_naive(df_base[fecha_col_base], dayfirst_hint=dayfirst_base)
    df_traza["_T1"] = _to_datetime_naive(df_traza[fecha_col_traza], dayfirst_hint=dayfirst_traza)

    # Filtra inválidos
    df_base  = df_base[df_base["_RUT"].ne("") & df_base["_T0"].notna()].copy()
    df_traza = df_traza[df_traza["_RUT"].ne("") & df_traza["_T1"].notna()].copy()

    # Orden estable
    df_base  = df_base.sort_values(["_RUT", "_T0"], kind="mergesort").reset_index(drop=True)
    df_traza = df_traza.sort_values(["_RUT", "_T1"], kind="mergesort").reset_index(drop=True)

    # Preparar columnas extra a traer de traza
    # Puedes pasar una tupla/lista de nombres o un dict {col_traza: nombre_salida}
    if isinstance(extra_cols_traza, dict):
        extra_cols_in  = [c for c in extra_cols_traza.keys() if c in df_traza.columns]
        rename_map_out = {k: v for k, v in extra_cols_traza.items() if k in df_traza.columns}
    else:
        extra_cols_in  = [c for c in extra_cols_traza if c in df_traza.columns]
        rename_map_out = {}

    parts = []
    traza_groups = {k: v for k, v in df_traza.groupby("_RUT", sort=True)}
    for rut, left in df_base.groupby("_RUT", sort=True):
        right = traza_groups.get(rut)
        left = left.sort_values("_T0", kind="mergesort")
        if right is None or right.empty:
            # si no hay traza para ese RUT, agrega NaT y NaN en extras
            asign = {"_T1": pd.NaT}
            for c in extra_cols_in:
                asign[rename_map_out.get(c, c)] = pd.NA
            left = left.assign(**asign)
        else:
            right = right.sort_values("_T1", kind="mergesort")
            right_cols = ["_T1"] + extra_cols_in
            right_sub = right[right_cols].rename(columns=rename_map_out)
            left = pd.merge_asof(
                left,
                right_sub,
                left_on="_T0",
                right_on="_T1",
                direction="forward",
                tolerance=pd.Timedelta(days=30),
                allow_exact_matches=False
            )
        parts.append(left)

    merged = pd.concat(parts, ignore_index=True)

    # Delta horas y flags
    delta_horas = (merged["_T1"] - merged["_T0"]).dt.total_seconds() / 3600.0
    merged["HORAS_HASTA_CONTACTO"] = delta_horas
    valid = delta_horas.notna()

    merged["FCR_VOZ_24"]  = (valid & (delta_horas <= 24)).astype(int)
    merged["FCR_VOZ_7D"]  = (valid & (delta_horas <= 7*24)).astype(int)
    merged["FCR_VOZ_30D"] = (valid & (delta_horas <= 30*24)).astype(int)

    merged["NEXT_CONTACTO_VOZ"] = merged["_T1"]

    # Limpieza
    out = merged.drop(columns=["_RUT", "_T1"], errors="ignore")
    return out


In [52]:
base_fcr_flags = add_fcr_voz_flags(
    base_fcr=base_fcr,                  # tu base principal
    traza_comercial_con=traza_comercial_con,   # tu traza de voz
    rut_col_base="rut",
    rut_col_traza="RUT_CLIENTE",
    fecha_col_base="StorageDate",
    fecha_col_traza="INTERVALO"
)


In [53]:
base_fcr_flags

,CustomerIdentity,StorageDate,FECHA_SIGUIENTE,DIAS_HASTA_REITERACION,FLAG_REITERACION_24,FLAG_REITERACION_7D,FLAG_REITERACION_30D,SequentialId,AgentIdentity,Status,Team,Tags,QueueTime,FirstResponseTime,AverageResponseTime,CustomerPhoneNumber,campaignId,rut,NEUTRA,Semana,Def_IBR,Flag_Outbound,MAIL,Flag_Dotacion_Konecta,Defle_meta,EMPRESA_PAIS,MARCA,_T0,RAMAFINAL,HORAS_HASTA_CONTACTO,FCR_VOZ_24,FCR_VOZ_7D,FCR_VOZ_30D,NEXT_CONTACTO_VOZ
0,fd3296ba-7479-43b6-92ba-ccf64ce02a76@tunnel.ms...,2025-07-23 16:55:49,NaN,NaN,0,0,0,2978178,edwin.sandoval%40e.vtr.cl@blip.ai,ClosedAttendant,comercial_fijo,"[""Visita TÃÂ©cnica reag/desi/incum""]",00:00:00,00:00:14,120.0,56945116376,Contratar Servicios,10001053-4,0,3,0,0,edwin.sandoval,1,0,KONECTA PERU,VTR,2025-07-23 16:55:49,AIPTVSOPTEC2WAY,49.338056,0,1,1,2025-07-25 18:16:06
1,054ba0b7-dfca-4125-9974-5a61d18c3c3d@tunnel.ms...,2025-07-09 15:50:31,2025-07-09 16:17:30,0.0,1,1,1,2911028,samuel.correavelez%40ext.clarovtr.cl@blip.ai,ClosedClientInactivity,tecnico_fijo,"[""inactividad""]",00:00:04,00:00:13,170.0,56979845812,Servicio Hogar,10002375-k,0,1,0,0,samuel.correavelez,0,0,ATENTO COLOMBIA,VTR,2025-07-09 15:50:31,ACOMERCIAL,42.945278,0,1,1,2025-07-11 10:47:14
2,054ba0b7-dfca-4125-9974-5a61d18c3c3d@tunnel.ms...,2025-07-09 16:17:30,2025-07-09 16:42:36,0.0,1,1,1,2911207,kelly.rueda%40e.vtr.cl@blip.ai,ClosedClientInactivity,tecnico_fijo,"[""inactividad""]",00:00:00,00:00:35,NaN,56979845812,Servicio Hogar,10002375-k,0,1,0,0,kelly.rueda,0,0,ATENTO COLOMBIA,VTR,2025-07-09 16:17:30,ACOMERCIAL,42.495556,0,1,1,2025-07-11 10:47:14
3,054ba0b7-dfca-4125-9974-5a61d18c3c3d@tunnel.ms...,2025-07-09 16:42:36,2025-07-09 17:00:46,0.0,1,1,1,2911353,henguevbel.alfonzoguevara%40e.vtr.cl@blip.ai,ClosedClientInactivity,tecnico_fijo,"[""inactividad""]",00:00:00,00:01:15,NaN,56979845812,Servicio Hogar,10002375-k,0,1,0,0,henguevbel.alfonzoguevara,1,0,KONECTA PERU,VTR,2025-07-09 16:42:36,ACOMERCIAL,42.077222,0,1,1,2025-07-11 10:47:14
4,054ba0b7-dfca-4125-9974-5a61d18c3c3d@tunnel.ms...,2025-07-09 17:00:46,2025-07-09 17:02:42,0.0,1,1,1,2911460,kelly.rueda%40e.vtr.cl@blip.ai,Transferred,tecnico_fijo,NaN,00:00:00,00:00:19,NaN,56979845812,Servicio Hogar,10002375-k,0,1,0,0,kelly.rueda,0,0,ATENTO COLOMBIA,VTR,2025-07-09 17:00:46,ACOMERCIAL,41.774444,0,1,1,2025-07-11 10:47:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110814,767a2896-f32f-4fb6-ad8d-2325fd068860@tunnel.ms...,2025-07-31 19:16:02,2025-07-31 21:33:51,0.0,1,1,1,3013398,paola.ortizramirez%40ext.clarovtr.cl@blip.ai,ClosedClientInactivity,comercial_fijo,"[""inactividad""]",00:01:45,00:00:11,NaN,56958024964,Excepciones,NaN,0,4,0,0,paola.ortizramirez,0,0,ATENTO COLOMBIA,VTR,2025-07-31 19:16:02,NaN,NaN,0,0,0,NaT
110815,b197f5ab-08f7-4189-ad85-44a6a3878110@tunnel.ms...,2025-07-31 20:10:48,NaN,NaN,0,0,0,3013713,Karlita.gallardaycruzate%40e.vtr.cl@blip.ai,ClosedClientInactivity,comercial_fijo,"[""inactividad""]",00:00:00,00:00:09,NaN,56928231292,Excepciones,NaN,0,4,0,0,karlita.gallardaycruzate,0,0,KONECTA PERU,VTR,2025-07-31 20:10:48,NaN,NaN,0,0,0,NaT
110816,73161c62-0aa7-41ee-be3d-4cc496ecae80@tunnel.ms...,2025-07-31 20:15:36,2025-07-31 20:32:24,0.0,1,1,1,3013741,juliana.ladinoladino%40e.vtr.cl@blip.ai,ClosedClientInactivity,comercial_fijo,"[""Explic. Otros Cobros/boleta""]",00:03:00,00:00:17,NaN,56944145224.0,Excepciones,NaN,0,4,0,1,juliana.ladinoladino,0,0,ATENTO COLOMBIA,VTR,2025-07-31 20:15:36,NaN,NaN,0,0,0,NaT
110817,c5263cfc-0aee-47f6-a408-bda717e75b69@tunnel.ms...,2025-07-31 20:35:12,NaN,NaN,0,0,0,3013857,SEBASTIAN.HENAOESPINOSA%40ext.clarovtr.cl@blip.ai,ClosedAttendant,tecnico_fijo,"[""2_Streaming_Activacion_Video""]",00:00:00,00:00:21,216.0,56998977722.0,Consulta Saldo,NaN,0,4,0,0,sebastian.henaoespinosa,0,0,ATENTO COLOMBIA,VTR,2025-07-31 20:35:12,NaN,NaN,0,0,0,NaT


In [54]:
# Eliminar columnas (seguro aunque no existan)
base_fcr_flags = base_fcr_flags.drop(columns=["HORAS_HASTA_CONTACTO", "_T0"], errors="ignore")

In [55]:
# Ahora sí, guardás el archivo
base_fcr_flags.to_csv(r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\FCR\Base_Consolidada_FCR_JUL.csv", index=False)

# FCR No Voz CLARO

In [ ]:
# Define the file path
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Dotaciones\CLARO\Dotación_CLARO_20250131.xlsx"
# Read the specific sheet named 'Dotacion' from the Excel file
dotacion = pd.read_excel(archivo, sheet_name='Dotacion')  # Specify the sheet name

In [ ]:
dotacion_filtradac = dotacion[
    (dotacion['ESTADO_CONTRATO'].str.lower() == 'vigente') &
    (dotacion['PLATAFORMA'].str.lower().isin(['postventa no voz'])) &
    (dotacion['CARGO'].str.lower() == 'asesor') &
    (dotacion['ESTADO_ASESOR'].str.lower().isin(['activo']))
]

In [ ]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
dotacion_filtradac['MAIL'] = dotacion_filtradac['CORREO_ELECTRONICO'].str.split('@').str[0]

In [ ]:
# Creamos versiones en minúscula del campo MAIL para el join
dotacion_filtradac['MAIL'] = dotacion_filtradac['MAIL'].str.lower()
base_fcr_m['MAIL'] = base_fcr_m['MAIL'].str.lower()

In [ ]:
# Hacer merge directo por MAIL
base_fcr_m = base_fcr_m.merge(
    dotacion_filtradac[['MAIL', 'EMPRESA_PAIS']], 
    on='MAIL', 
    how='left'
)

# Crear campo MARCA: 'VTR' si coincide, vacío si no
base_fcr_m['MARCA'] = base_fcr_m['EMPRESA_PAIS'].notna().map({True: 'CLARO', False: ''})

In [ ]:
# Eliminar filas con NaN en EMPRESA_PAIS
base_fcr_m = base_fcr_m[base_fcr_m['EMPRESA_PAIS'].notna()].copy()

In [ ]:
base_fcr_m